In [1]:
import numpy as np
import pandas as pd
import re
import string
import pickle

In [2]:
txt = 'great product. i like it'

In [3]:
with open ('../static/model/corpora/stopwords/english','r') as file:
    sw= file.read().splitlines()


In [4]:
with open('../static/model/model.pickle','rb') as f :
    model= pickle.load(f)

In [5]:
from nltk.stem import PorterStemmer
ps=PorterStemmer()


In [6]:
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation,'')
    return text

In [7]:
def preprocessing (text):
    data = pd.DataFrame([text], columns=['tweet'])
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(x.lower() for x in x.split()))
    data['tweet']=data['tweet'].apply(lambda x : " ".join(re.sub(r'^https?:\/\/.*[\r\n]*','',x,flags=re.MULTILINE)for x in x.split()))
    data['tweet']=data['tweet'].apply(remove_punctuations)
    data['tweet']=data['tweet'].str.replace('\d+','',regex=True)
    data['tweet']=data['tweet'].apply(lambda x:" ".join(x for x in x.split() if x not in sw))
    data['tweet']=data['tweet'].apply(lambda x: " ".join(ps.stem(x) for x in x.split()))
    return data['tweet']

In [8]:
vocab = pd.read_csv('../static/model/vocabluary.txt', header=None)
tokens= vocab[0].tolist()

In [9]:
def vectorizer(ds,vocabluary):
    vectorized_lst =[]

    for sentence in ds:
        sentence_lst = np.zeros(len(vocabluary))

        for i in range(len(vocabluary)):
            if vocabluary[i] in sentence.split():
                sentence_lst[i] = 1

        vectorized_lst.append(sentence_lst)
        
    vectorized_lst_new = np.asarray(vectorized_lst,dtype=np.float32)

    return vectorized_lst_new

In [10]:
def get_prediction(vectorized_txt):
    prediction= model.predict(vectorized_txt)
    if prediction==1:
        return 'negative'
    else:
        return 'positive'

In [19]:
txt = 'Best product i have ever seen'
preprocessed_txt=preprocessing(txt)
vectorized_txt=vectorizer(preprocessed_txt,tokens)
get_prediction(vectorized_txt)

'negative'